<a href="https://colab.research.google.com/github/Hongfruit/NLP_proj/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4%EC%9D%8C%EC%84%B1%EC%96%B4%EC%B2%98%EB%A6%AC_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**나는 할 수 있다 아자아자~~!!**

# AI에게 한국인만 읽을 수 있는 리뷰 가르치기 전체 맥락

**1. 토큰이 언어 모델에 들어가기 앞서 어순을 반영하기 위해 거치는 단계**
* 토큰의 n차원 벡터와 어순을 반영하는 n차원 벡터(position Encoding)를 더해준다.

    *why? 주어 you와 목적어 you가 다름을 표시하기 위함*


* 토큰의 발음을 표현하는 n차원 벡터(Phonetic Encoding)도 같이 더함


> 방법: 최대한 여러개의 영단어와 그 단어를 발음 기호로 전사한 쌍을 준비,
        각 발음 기호의 음운자질(Phonetic feature) 준비.
        **어쨋든 발음을 표현하는 벡터를 만든다. (나중에 자세히 설명)**


-----





**2.   잘 됐나 확인하기 위해 사전학습된 한국어-영어번역 모델을 사용해 파인튜닝 진행**


```
# Helsinki-NLP/opus-mt-ko-en
```
* 학습데이터?

> 한영번역데이터 중 구어체에 노이즈 추가해 활용

* 노이즈 추가?
> 랜덤하게 k개의 어절을 고르고, 그 어절의 초성이 평음이면 랜덤하게 거센 소리 또는 된소리로 변형
-----------------------



**3. 동일한 학습데이터를 가지고 기존 모델과 Phonetic Encoding을 추가한 모델의 학습 후 성능 비교**

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers
!pip install pytorch-transformers

In [ ]:
!pip install git+https://github.com/crlotwhite/KoG2P_fork.git

In [ ]:
import kog2p

kog2p.runKoG2P('모듈에 발음 특징 벡터를 생성하는 관련 함수 또는 클래스가 있다는 것을 나타냅니다.')

In [ ]:
!git clone https://github.com/yeounyi/PhoneticTranslation.git

In [ ]:
%cd /content/drive/MyDrive/korean-phonetic-vectors

In [ ]:
!ls -a

In [ ]:
!pip install hgtk

# Korean Phonetic Vectors

In [ ]:
import pandas as pd
import pickle
from itertools import product
from collections import Counter
import sys
import re
import numpy as np
from numpy import dot
from numpy.linalg import norm
from KoG2P.g2p import runKoG2P
from generate_phonetic_feature_vectors import *

In [ ]:
runKoG2P('박물관', 'KoG2P/rulebook.txt')

In [ ]:
df = pd.read_csv('data/ipa_feats.csv')
df.features = df.features.apply(lambda x: tuple(x.split()))
phone_feature_map = dict(df.values)
phone_feature_map['^'] = tuple(['bgn'])
phone_feature_map['$'] = tuple(['end'])
phone_feature_map

In [ ]:
with open ('feat2vec.pickle', 'rb') as fp:
    feat2vec = pickle.load(fp)

In [ ]:
def word2phonvec(word):
    """Takes a korean word and returns a phonetic vector of it
    """
    phones = runKoG2P(re.sub('[^가-힣]','', word), 'KoG2P/rulebook.txt')
    features = Counter(feature_bigrams(phones.split(), phone_feature_map))
    word_phonvec = np.zeros(512)
    count = 0
    for feature in features:
        word_phonvec += feat2vec[feature] * features[feature]
        count += features[feature]

    word_phonvec = word_phonvec / count if count > 0 else word_phonvec

    # 마지막에 음절 수 곱해주기
    return word_phonvec * len(word)

# PhoneticTranslation

In [ ]:
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
from MarianMTPhoneticModel import MarianMTPhoneticModel
from tqdm import tqdm
import re
import pickle
import pandas as pd
import torch
from KoG2P.g2p import runKoG2P
import hgtk
import random
from tqdm import tqdm
tqdm.pandas()


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ko-en")
model = MarianMTPhoneticModel.from_pretrained("Helsinki-NLP/opus-mt-ko-en")

In [ ]:
ko_vocab_dict = {}
for vocab in tqdm(tokenizer.encoder):
    if re.findall('[^가-힣_]', vocab) or vocab == '_':
        continue
    ko_vocab_dict[vocab] = tokenizer.encoder[vocab]

len(ko_vocab_dict) # 31307

list(ko_vocab_dict.keys())[:10] # ['_들', '_이', '_을', '_의', '_은', '_에', '_가', '_는', '_를', '_그']

vocab2phonvec = {}
for vocab in tqdm(ko_vocab_dict):
    vocab2phonvec[ko_vocab_dict[vocab]] = word2phonvec(vocab)

with open('vocab2phonvec.pickle', 'wb') as fp:
    pickle.dump(vocab2phonvec, fp)


In [ ]:
with open ('feat2vec.pickle', 'rb') as fp:
    feat2vec = pickle.load(fp)

with open ('vocab2phonvec.pickle', 'rb') as fp:
    vocab2phonvec = pickle.load(fp)

ipadf = pd.read_csv('data/ipa_feats.csv')
ipadf.features = ipadf.features.apply(lambda x: tuple(x.split()))
phone_feature_map = dict(ipadf.values)
phone_feature_map['^'] = tuple(['bgn'])
phone_feature_map['$'] = tuple(['end'])


In [ ]:
def get_phonetic_embedding(input_ids, sent):
    embed_phonetic = []
    if tokenizer.unk_token_id not in input_ids:
        for token_id in input_ids:
            if int(token_id) not in vocab2phonvec:
                embed_phonetic.append(torch.zeros(512))
            else:
                embed_phonetic.append(torch.from_numpy(vocab2phonvec[int(token_id)]))
    else:
        tokenized_sent = tokenizer.tokenize(sent)
        for token_id, token in zip(input_ids, tokenized_sent + ['<pad>'] * (len(input_ids) - len(tokenized_sent))):
            if token_id == tokenizer.unk_token_id:
                embed_phonetic.append(torch.from_numpy(word2phonvec(token)))
            elif int(token_id) in vocab2phonvec:
                embed_phonetic.append(torch.from_numpy(vocab2phonvec[int(token_id)]))
            else:
                embed_phonetic.append(torch.zeros(512))

    return torch.stack(embed_phonetic).float()

In [ ]:
phon_dict = {'ㄱ':['ㄲ','ㅋ'], 'ㄷ':['ㄸ','ㅌ'], 'ㅂ':['ㅃ','ㅍ'], 'ㅅ':['ㅆ'], 'ㅈ':['ㅉ','ㅊ']}

def add_noise(eumjeol):
    result = []
    for i,phoneme in enumerate(hgtk.letter.decompose(eumjeol)):
        # 종성은 변하지 않음
        if i == 2 or phoneme not in phon_dict:
            result.append(phoneme)
        else:
            result.append(random.choice(phon_dict[phoneme]))

    if len(result) == 2:
        return hgtk.letter.compose(result[0], result[1])
    else:
        try:
            return hgtk.letter.compose(result[0], result[1], result[2])
        except:
            print(eumjeol, result)
            return eumjeol

In [ ]:
def build_noise_data(sent):
    random_eojeols = random.choices(sent.split(), k=2)
    for random_eojeol in random_eojeols:
        if hgtk.checker.is_hangul(random_eojeol):
            noised_eojeol = ''.join([add_noise(eumjeol) for eumjeol in random_eojeol])
            sent = sent.replace(random_eojeol, noised_eojeol)
    return sent

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, ko_sents=None, en_sents=None, tokenizer=None, label_yn=True):
        self.label_yn = label_yn
        inputs = tokenizer(ko_sents, return_tensors="pt", padding=True, truncation=True)

        self.input_ids = []
        self.embed_phonetic = []
        for input_id, sent in zip(inputs['input_ids'], ko_sents):
            input_id = input_id.clone().detach()
            emb_pho = get_phonetic_embedding(input_id, sent)
            self.input_ids.append(input_id)
            self.embed_phonetic.append(emb_pho)
            assert len(input_id) == len(emb_pho)
        self.attention_mask = [i.clone().detach() for i in inputs['attention_mask']]

        if self.label_yn:
            labels = tokenizer(en_sents, return_tensors="pt", padding=True, truncation=True).input_ids
            self.labels = [i.clone().detach() for i in labels]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if self.label_yn:
            return {'input_ids': self.input_ids[idx], 'attention_mask': self.attention_mask[idx], \
                    'embed_phonetic': self.embed_phonetic[idx], 'labels': self.labels[idx]}
        return {'input_ids': self.input_ids[idx], 'attention_mask': self.attention_mask[idx], \
                    'embed_phonetic': self.embed_phonetic[idx]}

In [ ]:
df = pd.read_csv('mt-data.csv')
df

In [ ]:
!pip install accelerate -U

In [ ]:
df['noised_ko'] = df['원문'].apply(lambda x: build_noise_data(x))
df['en']=df['번역문']

msk = np.random.rand(len(df)) < 0.8

train_df = df[msk]
eval_df = df[~msk]

train_df.to_csv('train.csv', index=False, encoding='utf-8')
eval_df.to_csv('eval.csv', index=False, encoding='utf-8')

train_data = CustomDataset(train_df.noised_ko.tolist(), train_df.en.tolist(), tokenizer)
eval_data = CustomDataset(eval_df.noised_ko.tolist(), eval_df.en.tolist(), tokenizer)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# freeze original Bart Encoder, Decoder except the last layer
# for param in model.model.encoder.layers[:-2].parameters():
#     param.requires_grad = False
# for param in model.model.decoder.layers[:-1].parameters():
#    param.requires_grad = False


# +

training_args = TrainingArguments(
    'PhoneticTranslation-noise2-epoch15-ckpt',
    # load_best_model_at_end = True,
    num_train_epochs=15,  # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    # weight_decay=0.01,  # strength of weight decay
    # eval_accumulation_steps=1,
    save_steps = 999999999999999 # checkpoint 중간에 저장하지 말라고
)
# -

trainer = Trainer(
    model=model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=train_data,  # training dataset
    eval_dataset=eval_data,
    # compute_metrics= compute_metrics
)

trainer.train()

trainer.evaluate()

model.save_pretrained('./PhoneticTranslationModel-noise2-epoch15')


In [ ]:
def get_inference_batch(ko_sents):
        inputs = tokenizer(ko_sents, return_tensors="pt", padding=True, truncation=True)

        input_ids = []
        embed_phonetic = []
        for input_id, sent in zip(inputs['input_ids'], ko_sents):
            input_ids.append(input_id.clone().detach())
            embed_phonetic.append(get_phonetic_embedding(input_id, sent))
        attention_mask = [i.clone().detach() for i in inputs['attention_mask']]

        input_ids = torch.stack(input_ids)
        attention_mask = torch.stack(attention_mask)
        embed_phonetic = torch.stack(embed_phonetic).float()

        return {'input_ids':input_ids, 'attention_mask':attention_mask, 'embed_phonetic':embed_phonetic}


sample_texts = ['박물관이 어디 있나요?', "빡물관이 어디 있나요?", "빡물콴이 어디 있나요?", "완쩐 뼐루에요"]
batch = get_inference_batch(sample_texts)
generated_ids = model.generate(**batch)
translated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
translated_texts

for sample, result in zip(sample_texts, translated_texts):
    print(f'{sample} --> {result}')


In [ ]:
%cd ..

In [ ]:
!mv korean-phonetic-vectors/ drive/MyDrive/